In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import numpy as np
import os

In [ ]:
def show(imgs):
    for im in imgs:
        im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
        im = Image.fromarray(im)
        im.thumbnail((400,400))
        display(im)
    

imgs = [cv.imread(x) for x in [f"photos/{f}" for f in os.listdir("photos")]]
imgs = [(cv.resize(i,(0,0), i, 0.1, 0.1, cv.INTER_AREA),i) for i in imgs]

def thresh(img):

    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    # https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html
    x = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, int(gray.shape[1]/20)*2+1, 2)
    
    kernel = np.ones((3,3),np.uint8)
    x = cv.erode(x, kernel, iterations=1)
    
    return x

imgs = [(thresh(i),j) for i,j in imgs]

show([i for (i,j) in imgs])

In [ ]:
cards = []

def process(im,orig):
    # Nice contour docs: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html
    # Find all contours, sorted by size
    contours, hierarchy = cv.findContours(im, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_TC89_KCOS)
    contours.sort(key=cv.contourArea,reverse=True)
    
    # Simplify contours
    simple_contours = [cv.approxPolyDP(c, 0.05*cv.arcLength(c, True),True) for c in contours]
    
    # Keep contours whose simple shape roughly matches the original shape, is a rectangle, and whose area is at least 500.
    
    simple_contours = [s for s,c in zip(simple_contours, contours) if cv.matchShapes(c,s,cv.CONTOURS_MATCH_I1,0) < 0.2 and len(s) == 4 and cv.contourArea(s) > 500]
            
    mask = np.zeros(orig.shape[0:2], np.uint8)
    cv.fillPoly(mask, simple_contours,1)
    
    x = orig.copy()#cv.bitwise_and(orig, orig, mask=mask)
    
    cv.drawContours(x, [s*10 for s in simple_contours], -1, (0,255,255), 20)
    

    
    target = np.float32([[0,0], [0,200], [300,200], [300,0]])
    for s in simple_contours:
        s *= 10
        # Roll contour until we're at the start of a short side.
        s = np.roll(s, 2 * np.argmax([cv.arcLength(s[i:i+2], False) for i in range(3)]))
        
        s = np.float32(np.roll(s,2))
        M = cv.getPerspectiveTransform(s, target)
        persp = cv.warpPerspective(orig, M, (300,200))
        cards.append(persp)
    
    return x


show([process(i,j) for i,j in imgs])

In [ ]:
for i,im in enumerate(cards):
    im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    im = Image.fromarray(im)
    im.save(f"cards/card.{i}.jpg")
